# “挑战杯”大赛“揭榜挂帅”赛道-疲劳分神驾驶检测Baseline
本案例的第1、2步可以省略，第3步的文件夹中已包含处理好的数据集，如果不需要训练，可以直接运行步骤4<br>
注意：该案例中moxing库为华为自研，无法在本地安装。<br>
如需在本地使用案例代码，可以把代码拷贝之后 下载到本地使用<br>

## 1、视频抽帧
抽取关键帧图像使用代码如下，可能需要修改视频所在路径

这里只用了一开始提供的142段视频

In [1]:
import cv2
import argparse
import json
import os
import numpy as np
import errno



def getInfo(sourcePath):
    cap = cv2.VideoCapture(sourcePath)
    info = {
        "framecount": cap.get(cv2.CAP_PROP_FRAME_COUNT),
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "heigth": int(cap.get(cv2.CAP_PROP_FRAME_Heigth)),
        "codec": int(cap.get(cv2.CAP_PROP_FOURCC))
    }
    cap.release()
    return info


def scale(img, xScale, yScale):
    res = cv2.resize(img, None,fx=xScale, fy=yScale, interpolation = cv2.INTER_AREA)
    return res


def resize(img, width, heigth):
    res = cv2.resize(img, (width, heigth), interpolation=cv2.INTER_AREA)
    return res

#可能的优化：
#1.K-means clustering的可靠程度
#2.提取色块个数
#3.是否应该全局提取色块个数？

#对一张keyframe image用k-means clustering完成主要色块的bgr值和各色块在image中拥有的pixel个数
#返回cols:a list of {"count": 各色块在image中拥有的pixel个数, "col": 该色块的bgr值}
def extract_cols(image, numCols):
    # convert to np.float32 matrix that can be clustered
    Z = image.reshape((-1, 3))
    Z = np.float32(Z)

    #k-means clustering
    # Set parameters for the clustering
    max_iter = 20
    epsilon = 1.0
    K = numCols #cluster centroid个数
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, max_iter, epsilon)
    labels = np.array([]) #每个pixel属于哪个色块的编号的ordered list
    # cluster
    compactness, labels, centers = cv2.kmeans(Z, K, labels, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    #centers是

    clusterCounts = []#统计属于每一个色块的pixel个数的list
    for idx in range(K):
        count = len(Z[np.where(labels == idx)]) #属于某个色块的pixel个数
        clusterCounts.append(count)

    rgbCenters = []
    for center in centers:
        bgr = center.tolist()
        bgr.reverse()
        rgbCenters.append(bgr)

    cols = []
    for i in range(K):
        iCol = {
            "count": clusterCounts[i],
            "col": rgbCenters[i]
        }
        cols.append(iCol)

    return cols


#可能的优化：
#1.提取灰度信息可靠吗？
#2.提取灰度信息后
###1.缩放图片至四分之一会产生什么影响？
###2.对缩放后的gray进行了高斯模糊会产生什么影响？
#3.逐帧比较灰度点不同个数多少可以判定为

#返还data
#data组成：
##1."frame_info": a list of {"frame_number","与上一帧灰度像素点不同个数(diff_count)包括0个"}
##2."stats":{
'''
        "num": 帧数,
        "min": 灰度差异个数最小值,
        "max": 灰度差异个数最大值,
        "mean": 灰度差异个数平均值,
        "median": 灰度差异个数中位数,
        "sd": 灰度差异个数标准差,
        "greater_than_mean": 灰度差异个数比平均值大的帧数,
        "greater_than_median": 灰度差异个数比中位数大的帧数,
        "greater_than_one_sd": 灰度差异个数超出1个std的帧数,
        "greater_than_two_sd": 灰度差异个数超出2个std的帧数,
        "greater_than_three_sd": 灰度差异个数超出3个std的帧数,
'''
### }
def calculateFrameStats(sourcePath, verbose=True, after_frame=0):  # 提取相邻帧的差别

    cap = cv2.VideoCapture(sourcePath)  # 提取视频

    data = {
        "frame_info": []
    }

    lastFrame = None
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break

        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES) - 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)      # 提取灰度信息
        gray = scale(gray, 0.25, 0.25)      # 缩放为原来的四分之一
        gray = cv2.GaussianBlur(gray, (9, 9), 0.0)   # 做高斯模糊
        # lastFrame = gray
        if frame_number < after_frame:
            lastFrame = gray
            continue

        if lastFrame is not None:
            diff = cv2.subtract(gray, lastFrame)        # 用当前帧减去上一帧
            diffMag = cv2.countNonZero(diff)        # 计算两帧灰度值不同的像素点个数
            frame_info = {
                "frame_number": int(frame_number),
                "diff_count": int(diffMag)
            }
            data["frame_info"].append(frame_info)
            if verbose:
                cv2.imshow('diff', diff)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        # Keep a ref to this frame for differencing on the next iteration
        lastFrame = gray

    cap.release()
    cv2.destroyAllWindows()

    # compute some states
    diff_counts = [fi["diff_count"] for fi in data["frame_info"]]
    data["stats"] = {
        "num": len(diff_counts),
        "min": np.min(diff_counts),
        "max": np.max(diff_counts),
        "mean": np.mean(diff_counts),
        "median": np.median(diff_counts),
        "sd": np.std(diff_counts)   # 计算所有帧之间, 像素变化个数的标准差
    }
    greater_than_mean = [fi for fi in data["frame_info"] if fi["diff_count"] > data["stats"]["mean"]]
    greater_than_median = [fi for fi in data["frame_info"] if fi["diff_count"] > data["stats"]["median"]]
    greater_than_one_sd = [fi for fi in data["frame_info"] if fi["diff_count"] > data["stats"]["sd"] + data["stats"]["mean"]]
    greater_than_two_sd = [fi for fi in data["frame_info"] if fi["diff_count"] > (data["stats"]["sd"] * 2) + data["stats"]["mean"]]
    greater_than_three_sd = [fi for fi in data["frame_info"] if fi["diff_count"] > (data["stats"]["sd"] * 3) + data["stats"]["mean"]]

    # 统计其他信息
    data["stats"]["greater_than_mean"] = len(greater_than_mean)
    data["stats"]["greater_than_median"] = len(greater_than_median)
    data["stats"]["greater_than_one_sd"] = len(greater_than_one_sd)
    data["stats"]["greater_than_three_sd"] = len(greater_than_three_sd)
    data["stats"]["greater_than_two_sd"] = len(greater_than_two_sd)

    return data

def writeImagePyramid(destPath,  name, seqNumber, image):
    fullPath = os.path.join(destPath, name + "_" + str(seqNumber) + ".png")
    cv2.imwrite(fullPath, image)

#可能的优化
#1.灰度差异个数threshold取了mean+2.05*std可靠吗？
#2.喂给extract_cols的一帧的数据是放缩到了长宽都是100有影响吗？
###找的色块个数是5个合理吗？

#返还data
#给灰度差异个数超过threshold的所有keyframe添加了"dominant_cols"变成 {"frame_number","灰度像素点不同个数(diff_count)包括0个","dominant_cols"}

#把所有keyframe全部存进key frame/images 命名："day_man_001_10_1_202.png"

def detectScenes(sourcePath, destPath, data, name, verbose=False):
    destDir = os.path.join(destPath, "images")

    # TODO make sd multiplier externally configurable
    # diff_threshold = (data["stats"]["sd"] * 1.85) + data["stats"]["mean"]
    diff_threshold = (data["stats"]["sd"] * 2.05) + (data["stats"]["mean"]) #difference比均值不超过2.05个std

    cap = cv2.VideoCapture(sourcePath)
    for index, fi in enumerate(data["frame_info"]):
        if fi["diff_count"] < diff_threshold:
            continue

        cap.set(cv2.CAP_PROP_POS_FRAMES, fi["frame_number"])
        ret, frame = cap.read()

        # extract dominant color
        small = resize(frame, 100, 100)
        cols = extract_cols(small, 5)
        data["frame_info"][index]["dominant_cols"] = cols

        if frame is not None:
            # file_name = sourcePath.split('.')[0]
            writeImagePyramid(destDir, name, fi["frame_number"], frame)

            if verbose:
                cv2.imshow('extract', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()
    return data


def makeOutputDirs(path):
    try:
        os.makedirs(os.path.join(path, "metadata"))
        os.makedirs(os.path.join(path, "images"))

    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


In [ ]:
#import moxing as mox
#mox.file.copy_parallel('obs://obs-aigallery-zc/clf/dataset/Fatigue_driving_detection_video','Fatigue_driving_detection_video')  # 拷贝142段视频到ModelArts Notebook

抽帧 需要一段时间，请耐心等待

In [3]:
#对每一个视频进行
#1.calculateFrameStats =>得到基本data（针对每一帧得到与上一帧灰度像素点不同个数(diff_count)，并得到对于所有帧diff_count的一些stats量)
#2.detectScenes =>在基本data的基础上，基于diff_count>threshold得到keyframe并给每个keyframe进行5个色块提取，得到各个色块含pixel个数和bgr值存在dominant_cols中

#建立key frame文件夹
#key frame：
##1.images文件夹：
####1.文件名字：day_man_001_10_1_202.png
######内容：所有视频的所有keyframe images
##2.metadata文件夹：
####1.文件名字：day_man_001_10_1-keyframe-meta.txt
######内容：data全部
####2.文件名字：day_man_001_10_1-meta.txt
######内容：keyframeInfo

dest = "key frame" # 抽取图像保存路径
makeOutputDirs(dest)
test_path ='../Fatigue_driving_detection_video'   # 在这里修改视频路径
filenames = os.listdir(test_path)
count = 0
for filename in filenames:

    source = os.path.join(test_path, filename)

    name = filename.split('.')[0]

    data = calculateFrameStats(source, False, 0)
    data = detectScenes(source, dest, data, name, False)
    keyframeInfo = [frame_info for frame_info in data["frame_info"] if "dominant_cols" in frame_info]

    # Write out the results

    data_fp = os.path.join(dest, "metadata", name + "-meta.txt")
    with open(data_fp, 'w') as f:
        f.write(str(data))

    keyframe_info_fp = os.path.join(dest, "metadata", name + "-keyframe-meta.txt")
    with open(keyframe_info_fp, 'w') as f:
        f.write(str(keyframeInfo))
print(count)

KeyboardInterrupt: 

## 2、进行目标检测图像标注
我使用的是ModelArts标注 <br>
2.1 将上面的关键帧图像拷贝到你自己的OBS桶文件目录A中 <br>
2.2 点击[此处](https://console.huaweicloud.com/modelarts/?region=cn-north-4#/dataset)进入ModelArts数据管理页面，新建物体检测数据集，数据输入选择上述的OBS桶文件目录A，输出任意选择OBS桶内其他目录B <br>
2.3 开始标注，标注人脸标签face和手机标签phone（我标注的时候包括车内所有人脸），标注一部分（如200张图像）即可 <br>
2.4 开始智能标注，智能标注需要等待几十分钟，将智能标注后的图像进行核对确认 <br>
2.5 导出所有标注完的图像到OBS桶文件夹C中（尽量和A、B区分开） <br>
2.6 将OBS桶文件夹C中图像拷贝到ModelArts Notebook进行转换（因为训练所需为TXT标注格式、标注转换代码如下）<br>
2.7 划分训练集、验证集、测试集（划分代码如下）<br>


In [ ]:
'***转换xml标注文件为txt格式，无法直接运行***'
import copy
from lxml.etree import Element, SubElement, tostring, ElementTree

import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join

classes = ["face", "phone"]  # 类别


def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)


def convert_annotation(image_id):
    in_file = open('./label_xml/%s.xml' % (image_id), encoding='UTF-8')

    out_file = open('yolov7/datasets/Fatigue_driving_detection/txt_labels/%s.txt' % (image_id), 'w')  # 生成txt格式文件, 保存在yolov7训练所需的数据集路径中
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        print(cls)
        if cls not in classes:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

xml_path = './label_xml/'  # xml_path应该是上述步骤OBS桶文件夹C中的所有文件，记得拷贝过来

img_xmls = os.listdir(xml_path)
for img_xml in img_xmls:
    label_name = img_xml.split('.')[0]
    if  img_xml.split('.')[1] == 'xml':
        convert_annotation(label_name)

In [ ]:
'***转换xml标注文件为txt格式，无法直接运行***'     

#随机选择300张标注完的各种keyframe（不一定来自于同一段视频）做测试集sample
#随机选择300张标注完的各种keyframe（不一定来自于同一段视频）做验证集test
#不被选为sample且不被选为test的全都归于训练集train

#import moxing as mox
from random import sample
file_list = mox.file.list_directory( xml_path)  # xml_path中是上述步骤OBS桶文件夹C中的所有文件，记得拷贝到本地
print(len(file_list))
val_file_list = sample(file_list, 300)  # 选择了300张做测试集
line = ''
for i in val_file_list:
    if i.endswith('.png') :
        line += 'datasets/Fatigue_driving_detection/images/'+i+'\n'     # datasets/Fatigue_driving_detection/images/ 是yolov7训练使用的
with open('yolov7/datasets/Fatigue_driving_detection/val.txt', 'w+') as f:  
    f.writelines(line)

test_file_list = sample(file_list, 300)
line = ''
for i in test_file_list:
    if i.endswith('.png'):
        line += 'datasets/Fatigue_driving_detection/images/'+i+'\n'
with open('yolov7/datasets/Fatigue_driving_detection/test.txt', 'w+') as f:
    f.writelines(line)

line = ''
for i in file_list:
    if i not in val_file_list and i not in test_file_list:
        if i.endswith('.png') :
            line += 'datasets/Fatigue_driving_detection/images/'+i+'\n'
with open('yolov7/datasets/Fatigue_driving_detection/train.txt', 'w+') as f:
    f.writelines(line)

## 3、使用yolov7训练目标检测模型
点击[此处](https://developer.huaweicloud.com/develop/aigallery/notebook/detail?id=4f59e6c9-3e48-48e7-a723-ac972bb85aa8)，直接运行即可

## 4、结合视频实现疲劳/分神驾驶检测
详细见如下代码<br>
如果你第3步训练得到了更好的目标检测模型，将best.pt拷贝替换即可

In [3]:
# 拷贝提供的baseline 模型包到本地
#import moxing as mox
#mox.file.copy_parallel('obs://obs-aigallery-zc/clf/model/video_classification','video_classification')

In [ ]:
# 推理代码 展示
from PIL import Image
import copy
import sys
import traceback
import os
import numpy as np
import time
import cv2
from input_reader import InputReader
from tracker import Tracker
from EAR import eye_aspect_ratio
from MAR import mouth_aspect_ratio

from models.experimental import attempt_load
from utils1.general import check_img_size
from tempfile import NamedTemporaryFile
from utils1.torch_utils import TracedModel
from detect import detect
from model_service.pytorch_model_service import PTServingBaseService

import threading
from concurrent.futures import ThreadPoolExecutor


class fatigue_driving_detection(PTServingBaseService):
    def __init__(self, model_name, model_path):
        # these three parameters are no need to modify
        self.model_name = model_name
        self.model_path = model_path

        self.capture = 'test.mp4'
        self.result = 0
        self.cancel_flag = threading.Event()

        self.width = 1920
        self.height = 1080
        self.fps = 30
        self.first = True

        self.standard_pose = [180, 40, 80]
        self.look_around_frame = 0
        self.eyes_closed_frame = 0
        self.mouth_open_frame = 0
        self.use_phone_frame = 0
        # lStart, lEnd) = (42, 48)
        self.lStart = 42
        self.lEnd = 48
        # (rStart, rEnd) = (36, 42)
        self.rStart = 36
        self.rEnd = 42
        # (mStart, mEnd) = (49, 66)
        self.mStart = 49
        self.mEnd = 66
        self.EYE_AR_THRESH = 0.2
        self.MOUTH_AR_THRESH = 0.6
        self.frame_3s = self.fps * 3
        self.face_detect = 0

        self.weights = "best.pt"
        self.imgsz = 640

        self.device = 'cpu'  # 大赛后台使用CPU判分

        model = attempt_load(model_path, map_location=self.device)
        self.stride = int(model.stride.max())
        self.imgsz = check_img_size(self.imgsz, s=self.stride)

        self.model = TracedModel(model, self.device, self.imgsz)


        self.need_reinit = 0
        self.failures = 0

        self.tracker = Tracker(self.width, self.height, threshold=None, max_threads=4, max_faces=4,
                          discard_after=10, scan_every=3, silent=True, model_type=3,
                          model_dir=None, no_gaze=False, detection_threshold=0.6,
                          use_retinaface=0, max_feature_updates=900,
                          static_model=True, try_hard=False)

        # self.temp = NamedTemporaryFile(delete=False)  # 用来存储视频的临时文件

    def _preprocess(self, data):
        # preprocessed_data = {}
        for k, v in data.items():
            for file_name, file_content in v.items():
                try:
                    try:
                        with open(self.capture, 'wb') as f:
                            file_content_bytes = file_content.read()
                            f.write(file_content_bytes)

                    except Exception:
                        return {"message": "There was an error loading the file"}

                    # self.capture = self.temp.name  # Pass temp.name to VideoCapture()
                except Exception:
                    return {"message": "There was an error processing the file"}
        return 'ok'
    
    # 检测是否转头
    def checkLookAround(self,f):
        while not self.cancel_flag.is_set():
            if np.abs(self.standard_pose[0] - f.euler[0]) >= 45 or np.abs(self.standard_pose[1] - f.euler[1]) >= 45 or \
                np.abs(self.standard_pose[2] - f.euler[2]) >= 45:
                self.look_around_frame += 1
            else:
                self.look_around_frame = 0
            if self.look_around_frame >= self.frame_3s:
                self.result = 4
            return
    
    # 检测是否闭眼
    def checkClosedEyes(self,f):
        while not self.cancel_flag.is_set():
            leftEye = f.lms[self.lStart:self.lEnd]
            rightEye = f.lms[self.rStart:self.rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            # average the eye aspect ratio together for both eyes
            ear = (leftEAR + rightEAR) / 2.0
            if ear < self.EYE_AR_THRESH:
                self.eyes_closed_frame += 1
            else:
                self.eyes_closed_frame = 0
                # print(ear, eyes_closed_frame)
            if self.eyes_closed_frame >= self.frame_3s:
                self.result = 1
            return

    # 检测是否张嘴
    def checkMouthOpen(self,f):
        while not self.cancel_flag.is_set():
            mar = mouth_aspect_ratio(f.lms)
            if mar > self.MOUTH_AR_THRESH:
                self.mouth_open_frame += 1
            if self.mouth_open_frame >= self.frame_3s:
                self.result = 2
            return
        
        

    def _inference(self, data):
        """
        model inference function
        Here are a inference example of resnet, if you use another model, please modify this function
        """
        print(data)
        result = {"result": {"category": 0, "duration": 6000}}

        self.input_reader = InputReader(self.capture, 0, self.width, self.height, self.fps)
        source_name = self.input_reader.name
        now = time.time()
        while self.input_reader.is_open():

            if not self.input_reader.is_open() or self.need_reinit == 1:
                self.input_reader = InputReader(self.capture, 0, self.width, self.height, self.fps, use_dshowcapture=False, dcap=None)
                if self.input_reader.name != source_name:
                    print(f"Failed to reinitialize camera and got {self.input_reader.name} instead of {source_name}.")
                    # sys.exit(1)
                self.need_reinit = 2
                time.sleep(0.02)
                continue
            if not self.input_reader.is_ready():
                time.sleep(0.02)
                continue

            ret, frame = self.input_reader.read()

            self.need_reinit = 0

            try:
                if frame is not None:
                    # 剪裁主驾驶位
                    frame = frame[:, 600:1920, :]

                    # 检测驾驶员是否接打电话 以及低头的人脸
                    bbox = detect(self.model, frame, self.stride, self.imgsz)
                    # print(results)

                    for box in bbox:
                        if box[0] == 0:
                            self.face_detect = 1
                        if box[0] == 1:
                            self.use_phone_frame += 1

                    if self.use_phone_frame >= self.frame_3s:
                        self.result = 3
                        result['result']['category'] = 3
                        break

                    # 检测驾驶员是否张嘴、闭眼、转头
                    faces = self.tracker.predict(frame)
                    if len(faces) > 0:

                        face_num = 0
                        max_x = 0
                        for face_num_index, f in enumerate(faces):
                            if max_x <= f.bbox[3]:
                                face_num = face_num_index
                                max_x = f.bbox[3]



                        f = faces[face_num]
                        f = copy.copy(f)

                        pool = ThreadPoolExecutor(3)

                        lookAround = pool.submit(self.checkLookAround,self,f)
                        mouthOpen = pool.submit(self.checkMouthOpen,self,f)
                        eyeClosed = pool.submit(self.checkClosedEyes,self,f)

                        pool.shutdown()

                        while not self.result:
                            if (lookAround.done() and mouthOpen.done() and eyeClosed.done()):
                                break

                        if self.result:
                            self.cancel_flag.set()
                            result['result']['category'] = self.result
                            break

                    else:
                        if self.face_detect:
                            self.look_around_frame += 1
                            self.face_detect = 0
                            if self.look_around_frame >= self.frame_3s:
                                result['result']['category'] = 4
                                break
                    

                    self.failures = 0
                else:
                    break
            except Exception as e:
                if e.__class__ == KeyboardInterrupt:
                    print("Quitting")
                    break
                traceback.print_exc()
                self.failures += 1
                if self.failures > 30:   # 失败超过30次就默认返回
                    break
            del frame
        final_time = time.time()
        duration = int(np.round((final_time - now) * 1000))
        result['result']['duration'] = duration
        return result

    def _postprocess(self, data):
        os.remove(self.capture)
        return data

将模型导入到ModelArts,这里使用了ModelArts的SDK，你也可以在ModelArts控制台界面直接导入模型<br>
完成这一步，可以点击[此处](https://console.huaweicloud.com/modelarts/?region=cn-north-4#/myModel)，前往AI应用管理界面提交模型，如果还需要进行测试，可以部署在线服务进行测试<br>
导入需要几分钟时间，请耐心等待

In [7]:
'''
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import TransformerConfig,Params
#!pip install json5
import json5
import re
import traceback
import random

try:
    session = Session()
    config_path = 'video_classification/model/config.json' 
    if mox.file.exists(config_path):                                        # 判断一下是否存在配置文件，如果没有则不能导入模型
        model_location =  'video_classification/model'
        model_name = "video_classification"
        load_dict = json5.loads(mox.file.read(config_path))
        model_type = load_dict['model_type']
        re_name = '_'+str(random.randint(0,1000))
        model_name += re_name
        print("正在导入模型,模型名称：", model_name)
        model_instance = Model(
                     session, 
                     model_name=model_name,               # 模型名称
                     model_version="1.0.0",               # 模型版本
                      source_location_type='LOCAL_SOURCE',
                     source_location=model_location,      # 模型文件路径
                     model_type=model_type,               # 模型类型
                     )

    print("所有模型导入完成")
except Exception as e:
    print("发生了一些问题，请看下面的报错信息：") 
    traceback.print_exc()
    print("模型导入失败")
'''

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


















You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python3.7 -m pip install --upgrade pip' command.



正在导入模型,模型名称： video_classification_900



modelarts-cn-north-4-72b6beb2 is existed


Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147/model



Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147



Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147



Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147



Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147



Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-0

Successfully upload file /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to OBS modelarts-cn-north-4-72b6beb2/model-0331-020147



Successfully upload model files from /home/ma-user/work/ma_share/Fatigue_driving_detection/video_classification/model to obs path /modelarts-cn-north-4-72b6beb2/model-0331-020147.



The model source location is https://modelarts-cn-north-4-72b6beb2.obs.cn-north-4.myhuaweicloud.com/model-0331-020147/model



publishing



published



所有模型导入完成
